In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,DateType
import os
import logging
import configparser

In [2]:
spark = SparkSession.builder \
    .appName("appGlob") \
    .config("spark.jars","../controller/postgresql-42.7.2.jar")\
    .config('spark.driver.extraClassPath', '../controller/postgresql-42.7.2.jar') \
    .getOrCreate()

In [3]:
config = configparser.ConfigParser()
config.read("../properties.conf")
database_config = config["database"]



In [4]:
spark.version

'3.5.0'

In [5]:
#log
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("ReadCSV")


In [6]:
#LOAD_FILE
filePath = "C:/Users/andre/OneDrive/Escritorio/data_challenge_files/"
fileType = "hired_employees.csv"
# Define the schema
dp_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("date", DateType(), True),
    StructField("id_job", IntegerType(), True),
    StructField("id_department", IntegerType(), True)
])


In [7]:
try: 
    df = spark.read.csv(filePath+fileType,header=False, inferSchema=False, schema=dp_schema ,sep=",")
    
except Exception as e:
    logger.error("Error: %s", str(e), exc_info=True)
    df = spark.createDataFrame([], schema=schema)

In [8]:
logger.info("Esquema del DataFrame:")
df.printSchema()

INFO:ReadCSV:Esquema del DataFrame:


root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- date: date (nullable = true)
 |-- id_job: integer (nullable = true)
 |-- id_department: integer (nullable = true)



In [11]:
df.count()

1999

In [13]:
try :
    df.write \
    .format("jdbc") \
    .mode("overwrite") \
    .option("url", database_config["url"]) \
    .option("dbtable","employees") \
    .option("user", database_config["user"]) \
    .option("password", database_config["password"]) \
    .option("driver", database_config["driver"]) \
    .save()
except Exception as e:
    logger.error("Erro: review eschemas or datatypes %s", str(e), exc_info=True)